In [1]:
import pandas as pd
import numpy as np

In [2]:
target_output=pd.read_excel('recipe_to_load.xlsx')

b_r_birch=pd.read_excel('beverage_recipe_birchstreet.xlsx',header=5)
b_r_birch = b_r_birch[b_r_birch['Recipe type'] == 'Recipe']

b_i_birch=pd.read_excel('ingredient_masterlist_birchsreet.xlsx')

c_i_birch=pd.read_excel('Category_ingredient_BirchStreet.xlsx')
agbsrc=pd.read_excel('agbcode_file.xlsx')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
agbsrc=agbsrc[['Code AGB', 'WiseFins FR Category','WiseFins EN Category','WiseFins FR Subcategory','WiseFins EN Subcategory']]

agbsrc.drop_duplicates(inplace=True)

agbsrc['Code AGB']=agbsrc['Code AGB'].astype(str)

In [5]:
b_r_birch['resto'] = 'birchstreet'

b_r_birch['recipeGroup'] = 'Beverage'


b_r_birch['photo'] = 'no'

b_r_birch['cost'] = np.nan
b_r_birch['InventoryCode'] = np.nan
b_r_birch['portion'] = 1
b_r_birch['subRecipe'] = np.nan

b_r_birch.rename(columns={'Ingredient/Subrecipe name': 'recipeCompose'}, inplace=True)

b_r_birch.rename(columns={'Dish/Recipe name': 'recipeName'}, inplace=True)

b_r_birch.rename(columns={'UOM': 'unit'}, inplace=True)

b_r_birch['quantityAfter'] = b_r_birch['Quantity'].copy()

In [6]:
# Function to extract the first 8 characters from the 'recipeCompose' column
def extract_first_8_chars(recipe):
    return recipe[:8]

b_r_birch['SKU'] = b_r_birch['recipeCompose'].apply(extract_first_8_chars)
b_r_birch['SKU'] = b_r_birch['SKU'].astype(str)
b_i_birch['Supplier SKU']=b_i_birch['Supplier SKU'].astype(str)

In [7]:
# Calculate the average price for each unique code
b_i_birch = b_i_birch.groupby('Supplier SKU', as_index=False)['Unit price'].mean()

In [8]:
b_i_birch['Supplier SKU']=b_i_birch['Supplier SKU'].astype(str)

In [9]:
b_r_birch = b_r_birch.merge(b_i_birch[['Supplier SKU', 'Unit price']], how='left', left_on='SKU', right_on='Supplier SKU')

In [10]:
b_r_birch.drop_duplicates(inplace=True)

In [11]:
c_i_birch=c_i_birch[['Part #','AGBCode']]

In [12]:
c_i_birch.drop_duplicates(inplace=True)

In [13]:
b_r_birch = b_r_birch.merge(c_i_birch[['AGBCode','Part #']], how='left', left_on='SKU', right_on='Part #')

In [14]:
b_r_birch['AGBCode'].fillna(0, inplace=True)

# Convert the column to integer
b_r_birch['AGBCode'] = b_r_birch['AGBCode'].astype(str)

In [15]:
agbsrc['Code AGB'] = agbsrc['Code AGB'].str.strip()
b_r_birch['AGBCode'] = b_r_birch['AGBCode'].str.strip()

In [16]:
b_r_birch = b_r_birch.merge(agbsrc[['Code AGB','WiseFins EN Category','WiseFins EN Subcategory']], how='left', left_on='AGBCode', right_on='Code AGB')

In [17]:
b_r_birch.rename(columns={'WiseFins EN Category': 'category','WiseFins EN Subcategory': 'subcategory'}, inplace=True)

In [18]:
column_order = ['resto', 'subRecipe', 'recipeGroup', 'recipeName', 'recipeCompose', 'AGBCode','Code AGB', 'Quantity', 'cost', 'InventoryCode', 'unit', 'quantityAfter', 'photo','category','subcategory','portion','SKU']
# Reorder the columns
b_r_birch = b_r_birch[column_order]

b_r_birch.to_excel("output_b_r_birch.xlsx", index=False)

In [19]:
sales_a=pd.read_excel('output_menu_sales_analysis_transformed.xlsx')

In [20]:
sales_a.head()

,Code,Name,Department,Course,Qty.,Menu Mix %,Sales Price,Unit Cost,Cost %,Discount Amount,CM,Menu Revenue,Menu Cost,Menu CM,Menu CM %,CM Category,Menu Mix Category,Classification,recipeGroupParent
0,SINRS-19250002,Open Food Tastingchinese food tasting,FOOD,NaN,165.0,0.491071,207.454545,0,0,-34230.0,207.454545,34230.0,0,34230.0,0.686605,HIGH,HIGH,STARS,FOOD
1,SINRS-19250002,Open Food Tastingwedding food tasting,FOOD,NaN,108.0,0.321429,70.000000,0,0,-8540.0,70.000000,7560.0,0,7560.0,0.151643,LOW,HIGH,PLOWHORSES,FOOD
2,SINRS-19250002,Open Food TastingFOOD TASTING,FOOD,NaN,51.0,0.151786,70.000000,0,0,-2590.0,70.000000,3570.0,0,3570.0,0.071609,LOW,HIGH,PLOWHORSES,FOOD
3,SINRS-19250002,Open Food Tastingwedding foodtasting,FOOD,NaN,14.0,0.041667,70.000000,0,0,-980.0,70.000000,980.0,0,980.0,0.019657,LOW,LOW,DOGS,FOOD
4,SINRS-19250002,Open Food Tastingindian food tasting,FOOD,NaN,6.0,0.017857,70.000000,0,0,-420.0,70.000000,420.0,0,420.0,0.008425,LOW,LOW,DOGS,FOOD


In [21]:
def extract_last_8_chars(recipe):
    return recipe[-8:]

sales_a['code_grpparent'] = sales_a['Code'].apply(extract_last_8_chars)
b_r_birch['code_grp']=b_r_birch['recipeName'].apply(extract_last_8_chars)

In [22]:
sales_a['code_grpparent']=sales_a['code_grpparent'].astype(str)
b_r_birch['code_grp']=b_r_birch['code_grp'].astype(str)

In [23]:
b_r_birch = b_r_birch.merge(sales_a[['Sales Price','recipeGroupParent','code_grpparent']], how='left', left_on='code_grp', right_on='code_grpparent')

In [24]:
b_r_birch.rename(columns={'Sales Price': 'selling'}, inplace=True)

In [25]:
b_r_birch['selling'] = b_r_birch['selling'].fillna(0.001)

In [26]:
column_order = ['resto', 'subRecipe', 'recipeGroup', 'recipeGroupParent', 'recipeName', 'recipeCompose', 'AGBCode','Quantity', 'cost', 'InventoryCode', 'unit', 'quantityAfter','portion', 'photo','selling','category','subcategory']
# Reorder the columns
b_r_birch = b_r_birch[column_order]

In [27]:
b_r_birch.head()

,resto,subRecipe,recipeGroup,recipeGroupParent,recipeName,recipeCompose,AGBCode,Quantity,cost,InventoryCode,unit,quantityAfter,portion,photo,selling,category,subcategory
0,birchstreet,NaN,Beverage,NaN,Cocktail - Vodkatini Martini 20000002,L265.014 Noilly Prat Ex Dry1000ml,0,10.0,NaN,NaN,ML,10.0,1,no,0.001,NaN,NaN
1,birchstreet,NaN,Beverage,NaN,Cocktail - Vodkatini Martini 20000002,L255.001 Absolute Blue BtEach,1008,90.0,NaN,NaN,ML,90.0,1,no,0.001,alcoholic beverage,spirits
2,birchstreet,NaN,Beverage,COCKTAIL,Cocktail - Bombay Martini Martini 20000005,L265.014 Noilly Prat Ex Dry1000ml,0,5.0,NaN,NaN,ML,5.0,1,no,22.000,NaN,NaN
3,birchstreet,NaN,Beverage,COCKTAIL,Cocktail - Bombay Martini Martini 20000005,"L257.014 Gin Beefeater,700mlBtl",1002,90.0,NaN,NaN,ML,90.0,1,no,22.000,alcoholic beverage,spirits
4,birchstreet,NaN,Beverage,NaN,Cocktail - Apple Truffle MARTINI_HP 20000007,"L257.001 Gin Beefeater 24700ml, Each",1002,45.0,NaN,NaN,ML,45.0,1,no,0.001,alcoholic beverage,spirits


In [28]:
b_r_birch.to_excel("output_birchstreet.xlsx", index=False)